In [12]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("k").getOrCreate()

24/04/26 09:50:33 WARN Utils: Your hostname, kyn resolves to a loopback address: 127.0.1.1; using 10.0.250.224 instead (on interface wlp0s20f3)
24/04/26 09:50:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/26 09:50:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [13]:
spark

In [14]:
df_spark=spark.read.csv("test4.csv",header=True,inferSchema=True)
df_spark.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [15]:
df_spark.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [16]:
df_spark.columns

['Name', 'age', 'Experience', 'Salary']

In [17]:
#vectorassembler for grouping 0r combining
from pyspark.ml.feature import VectorAssembler
feature=VectorAssembler(inputCols=["age","Experience"],
                        outputCol="Independent feature")
output=feature.transform(df_spark)
output.show()

+---------+---+----------+------+-------------------+
|     Name|age|Experience|Salary|Independent feature|
+---------+---+----------+------+-------------------+
|    Krish| 31|        10| 30000|        [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|         [30.0,8.0]|
|    Sunny| 29|         4| 20000|         [29.0,4.0]|
|     Paul| 24|         3| 20000|         [24.0,3.0]|
|   Harsha| 21|         1| 15000|         [21.0,1.0]|
|  Shubham| 23|         2| 18000|         [23.0,2.0]|
+---------+---+----------+------+-------------------+



In [18]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent feature']

In [19]:
finalize=output.select("Independent feature","Salary")
finalize.show()

+-------------------+------+
|Independent feature|Salary|
+-------------------+------+
|        [31.0,10.0]| 30000|
|         [30.0,8.0]| 25000|
|         [29.0,4.0]| 20000|
|         [24.0,3.0]| 20000|
|         [21.0,1.0]| 15000|
|         [23.0,2.0]| 18000|
+-------------------+------+



In [20]:
#linear regresssion
from pyspark.ml.regression import LinearRegression
#split
train_data,test_data=finalize.randomSplit([0.75,0.25])
#implement model
regressor=LinearRegression(
    featuresCol="Independent feature",
    labelCol="Salary"
)
#fit model
regressor=regressor.fit(train_data)

24/04/26 09:50:40 WARN Instrumentation: [b67eb33b] regParam is zero, which might cause numerical instability and overfitting.


In [21]:
regressor.coefficients

DenseVector([28.4757, 1271.3568])

In [22]:
regressor.intercept

14299.832495812996

In [23]:
pred_results=regressor.evaluate(test_data)
#predict model
pred_results.predictions.show()

+-------------------+------+------------------+
|Independent feature|Salary|        prediction|
+-------------------+------+------------------+
|        [31.0,10.0]| 30000|27896.147403685147|
+-------------------+------+------------------+



In [31]:
from pyspark.ml.feature import VectorAssembler
a=VectorAssembler(
    inputCols=["age","Experience"],
    outputCol="Independent feature")
b=a.transform(df_spark)
b.show()


+---------+---+----------+------+-------------------+
|     Name|age|Experience|Salary|Independent feature|
+---------+---+----------+------+-------------------+
|    Krish| 31|        10| 30000|        [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|         [30.0,8.0]|
|    Sunny| 29|         4| 20000|         [29.0,4.0]|
|     Paul| 24|         3| 20000|         [24.0,3.0]|
|   Harsha| 21|         1| 15000|         [21.0,1.0]|
|  Shubham| 23|         2| 18000|         [23.0,2.0]|
+---------+---+----------+------+-------------------+



In [36]:
finalize=b.select("Salary","Independent feature")
finalize.show()

+------+-------------------+
|Salary|Independent feature|
+------+-------------------+
| 30000|        [31.0,10.0]|
| 25000|         [30.0,8.0]|
| 20000|         [29.0,4.0]|
| 20000|         [24.0,3.0]|
| 15000|         [21.0,1.0]|
| 18000|         [23.0,2.0]|
+------+-------------------+



In [38]:
from pyspark.ml.regression import LinearRegression
train_data,test_data=finalize.randomSplit([0.75,0.25])
reg=LinearRegression(
    featuresCol="Independent feature",
    labelCol="Salary"
)
reg=reg.fit(train_data)


24/04/26 09:57:06 WARN Instrumentation: [c3a3d75c] regParam is zero, which might cause numerical instability and overfitting.


In [41]:
reg.coefficients

DenseVector([172.4138, 1206.8966])

In [42]:
reg.intercept

10172.41379310354

In [46]:
x=reg.evaluate(test_data)
x.predictions.show()

+------+-------------------+------------------+
|Salary|Independent feature|        prediction|
+------+-------------------+------------------+
| 18000|         [23.0,2.0]|16551.724137931044|
| 20000|         [24.0,3.0]|17931.034482758627|
| 30000|        [31.0,10.0]|27586.206896551732|
+------+-------------------+------------------+

